In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [24]:
pip install unstructured

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/4b/4c/0ebe18639be2a6b8e74dc60bf81fd4532289c80a0feac88004c28c7f1e0d/unstructured-0.10.25-py3-none-any.whl.metadata
  Obtaining dependency information for rapidfuzz from https://files.pythonhosted.org/packages/76/dc/158b4f30a50ef7c283c7834419928da6400ed299ccf69dfc653d5c4f20fa/rapidfuzz-3.4.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 1.9 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/1.7 MB 3.5 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.7 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------  1.8/1.8 MB 3

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [37]:
pip install jq

Defaulting to user installation because normal site-packages is not writeable
  Using cached jq-1.6.0.tar.gz (2.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build jq
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  Executing: ./configure CFLAGS=-fPIC --prefix=C:\Users\Administrator\AppData\Local\Temp\pip-install-6j82es9e\jq_22d57f4bdc834884bbbb6e28467b8470\_deps\build\onig-install-6.9.8
  error: [WinError 2] 系统找不到指定的文件。
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for jq
ERROR: Could not build wheels for jq, which is required to install pyproject.toml-based projects


In [4]:
loader = DirectoryLoader('E:/QA', glob="**/*.txt",  show_progress=True,use_multithreading=True, loader_kwargs={'encoding': 'utf-8'})
docs = loader.load()

100%|██████████| 1/1 [00:06<00:00,  6.28s/it]


In [50]:
len(docs)

1

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(docs)

In [6]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin


In [7]:
embeddings = GPT4AllEmbeddings()

gpt4all_falcon_model = "E:/ggml-model-gpt4all-falcon-q4_0.bin"

llm = GPT4All(model=gpt4all_falcon_model ,max_tokens=2048)

Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin


In [8]:
question="Which program does Suellen reference that shares its name with a snake?"

In [9]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(),
                                       return_source_documents=True)
result = qa_chain({"query": question})

print(f"Number of source documents: {len(result['source_documents'])}\n")

Number of source documents: 4



In [10]:
result['result']

' The program Sueleln references that shares its name with a snake is "Snake Nation".'

In [11]:
for i, doc in enumerate(result['source_documents'], 1):
    print(f"Document {i}:")
    print(f"Source: {doc.metadata['source']}")
    print(f"Content: {doc.page_content}\n")

Document 1:
Source: E:\QA\article_segment_sample.txt
Content: "id": "newsidal-NPR-162-27",

"speaker": "NEAL CONAN, HOST",

"text": "Tom Parkinson, you think a pattern would indicate - well, a pattern - if you're dispensing large numbers of pills to large numbers of people, maybe people should get suspicious."

},

{

"id": "newsidal-NPR-162-28",

"speaker": "TOM PARKINSON, BYLINE",

Document 2:
Source: E:\QA\article_segment_sample.txt
Content: "text": "But we're going to be joined, as well, by Greg McGuff. He's a division president for Southern California - in Southern California for Lennar Homes. The company saw this trend toward more family members living under the same roof as a bright spot in an otherwise gloomy housing market. Greg McGuff is with us now by phone from his office in Corona, California. Welcome to you."

},

{

"id": "newsidal-NPR-159-105",

"speaker": "GREG MCGUFF",

Document 3:
Source: E:\QA\article_segment_sample.txt
Content: "text": "And remind us about the Nati

In [38]:
from sklearn.metrics import f1_score
import numpy as np

chunk_sizes = [200, 400, 800, 1600]
overlaps = [0, 0.1, 0.2, 0.3]

def calculate_em(reference, prediction):
    return int(reference == prediction)

em_results = {}
f1_results = {}
text_results=[]

loader = DirectoryLoader('E:/QA', glob="**/*.txt", show_progress=True, use_multithreading=True, loader_kwargs={'encoding': 'utf-8'})
docs = loader.load()

question_list=["Which party does Obama belong to?","Who is the Seattle lawyer and Democratic superdelegate?","Who is a division president for Southern California?","Amazon was founded by whom?"]
answer_list=["Democratic Party","David McDonald","Greg McGuff","Jeff Bezos"]

for chunk_size in chunk_sizes:
    for overlap in overlaps:   
        result_list=[]
        boolean_list=[]
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
        all_splits = text_splitter.split_documents(docs)

        vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())
        llm = GPT4All(model="E:/ggml-model-gpt4all-falcon-q4_0.bin", max_tokens=2048)
        qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), return_source_documents=True)
        
        for question in question_list:
            result = qa_chain({"query": question})
            result_list.append(result)

        for i in range(4):
            boolean_list.append(result_list[i]==answer_list[i])
            
        f1 = f1_score([1,1,1,1], boolean_list, average='micro')

        em_results[(chunk_size, overlap)] = em
        f1_results[(chunk_size, overlap)] = f1
        text_results.append(result_list)

print("\nF1 Results:")
for config, f1 in f1_results.items():
    print(f"Chunk Size: {config[0]}, Overlap: {config[1]}, F1: {f1}")


100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin
Found model file at  C:\\\\Users\\

In [52]:
for i in range(16):
    for j in range(4):
        print(text_results[i][j]['result'])
    print('\n')

 Barack Obama belongs to the Democratic Party.
 The Seattle lawyer and Democratic superdelegate is David McDonald.
 Greg McGuff is the division president for Southern California.
 Amazon was founded by Jeff Bezos.


 Barack Obama belongs to the Democratic Party.
 The Seattle lawyer and Democratic superdelegate is David McDonald.
 Greg McGuff is the division president for Southern California.
 Amazon was founded by Jeff Bezos.


 Barack Obama belongs to the Democratic Party.
 The Seattle lawyer and Democratic superdelegate is David McDonald.
 Greg McGuff is the division president for Southern California.
 Amazon was founded by Jeff Bezos.


 Barack Obama belongs to the Democratic Party.
 The Seattle lawyer and Democratic superdelegate is David McDonald.
 Greg McGuff is the division president for Southern California.
 Amazon was founded by Jeff Bezos.


 Barack Obama belongs to the Democratic Party.
 The Seattle lawyer and Democratic superdelegate is David McDonald.
 Greg McGuff is the d